### Task 1
My Model data will be the Embedded Data since they got the same attribbutes as each mclimate report will share a date which got a oen-to-many relantionship between climate report and hotspot report, Climate report will has a field which contains an array of Hotspot report given the same date which is the foreign key for the hotspot report whihc climate report will contains general information of the specific date, while hotspot will be generated when a specific event is occured, so to enhance the relantionship ebtween them, the embedded data model is used.

In [1]:
import pymongo
import csv
from pprint import pprint
from pymongo import MongoClient

In [2]:
client = MongoClient("192.168.1.105", 27017) 


In [3]:

def getcolname(filename):
    with open(filename, 'r') as csvfile:
  
        reader = csv.reader(csvfile)
        column_names = next(reader) # get the column name as colnames is in the first row]
        print(column_names)
    return column_names

In [4]:
chis_cn=getcolname('climate_historic.csv')
rhis_cn=getcolname('hotspot_historic.csv') # get thec okname

['station', 'date', 'air_temperature_celcius', 'relative_humidity', 'windspeed_knots', 'max_wind_speed', 'precipitation', 'GHI_w/m2']
['latitude', 'longitude', 'datetime', 'confidence', 'date', 'surface_temperature_celcius']


In [5]:
db = client.fit3182

In [6]:

import json

In [7]:
climate_db= db["climate_db"]
climate_db.drop()
def collectClimate(filename):
   
    with open(filename, 'r') as csvfile:
        data = csv.DictReader(csvfile)
        prep_data = []
        for row in data: #convert to float to ease of aggregate
            for col_name in ['air_temperature_celcius', 'relative_humidity', 'windspeed_knots', 'max_wind_speed',"GHI_w/m2"]:
                row[col_name] = float(row[col_name])
                
            row['hotspot'] = []
           
            prep_data.append(row)

      
        climate_db.insert_many(prep_data) # insert many data as one

In [8]:
def collectHotSpot(filename):
  
    #collect hotspot based of filename
    with open(filename, 'r') as csvfile:
        
        data = csv.DictReader(csvfile)
        hotspot = []
        for row in data:
            for col_name in ['confidence', 'latitude', 'longitude','surface_temperature_celcius' ]:

                row[col_name]=float(row[col_name] ) 
            climate_db.update_one( {"date":row['date']}, {"$push":{"hotspot":row }}) # update one tiem at each time
            

In [9]:
collectClimate('climate_historic.csv') # collect climate report

In [10]:
collectHotSpot('hotspot_historic.csv') #collect hotspot report

In [11]:



documents = climate_db.find() #for each document

    # Print each document
for document in documents:
     
       pass


# Question 2


In [31]:
#2(a)
result2a= climate_db.find({"date": "12/12/2023" }) #find the data based on "12/12/2023" in date attribute

In [32]:
for result in result2a:
        print(result)

{'_id': ObjectId('6650a001757d21f561ed00f8'), 'station': '948702', 'date': '12/12/2023', 'air_temperature_celcius': 19.0, 'relative_humidity': 55.3, 'windspeed_knots': 6.2, 'max_wind_speed': 12.0, 'precipitation': '0.00I', 'GHI_w/m2': 156.0, 'hotspot': [{'latitude': -37.903, 'longitude': 145.25, 'datetime': '2023-12-12 00:45:38', 'confidence': 53.0, 'date': '12/12/2023', 'surface_temperature_celcius': 44.0}]}


In [26]:
#2(b)

#ffind confidence less than 100 and 65 and project attributes mentioned
result2b=climate_db.aggregate([{'$unwind': '$hotspot'},
                                     {'$match': {'hotspot.surface_temperature_celcius': {'$gte': 65,'$lte': 100}}},
                                    {'$project': {"_id":0,'latitude': '$hotspot.latitude','longitude': '$hotspot.longitude', 'confidence':'$hotspot.confidence',
                                                  'surface temperature': '$hotspot.surface_temperature_celcius'}}])
for doc in result2b:
    pprint(doc)

{'confidence': 94.0,
 'latitude': -37.2284,
 'longitude': 147.9187,
 'surface temperature': 73.0}
{'confidence': 97.0,
 'latitude': -37.6572,
 'longitude': 142.0703,
 'surface temperature': 80.0}
{'confidence': 84.0,
 'latitude': -37.0193,
 'longitude': 148.1459,
 'surface temperature': 71.0}
{'confidence': 100.0,
 'latitude': -37.4229,
 'longitude': 147.027,
 'surface temperature': 99.0}
{'confidence': 80.0,
 'latitude': -37.0055,
 'longitude': 148.1582,
 'surface temperature': 68.0}
{'confidence': 85.0,
 'latitude': -37.4128,
 'longitude': 147.0242,
 'surface temperature': 98.0}
{'confidence': 90.0,
 'latitude': -34.357,
 'longitude': 141.5361,
 'surface temperature': 67.0}
{'confidence': 93.0,
 'latitude': -34.3539,
 'longitude': 141.5547,
 'surface temperature': 72.0}
{'confidence': 90.0,
 'latitude': -36.9939,
 'longitude': 148.2244,
 'surface temperature': 68.0}
{'confidence': 95.0,
 'latitude': -36.9959,
 'longitude': 148.2118,
 'surface temperature': 75.0}
{'confidence': 89.0,


In [15]:
#2(c)

result2c = climate_db.aggregate([
    {"$unwind": "$hotspot"},
    {"$match": {
        "$or": [
            {"date": "15/12/2023"},
            {"date": "16/12/2023"}
        ]
    }},
    {"$project": {"_id": 0,"date": "$date",
        "surface temperature": "$hotspot.surface_temperature_celcius",
        "air temperature": "$air_temperature_celcius",
        "relative humidity": "$relative_humidity",
        "max wind speed": "$max_wind_speed"
    }}
])

for row in result2c:
    pprint(row)

{'air temperature': 18.0,
 'date': '15/12/2023',
 'max wind speed': 14.0,
 'relative humidity': 52.0,
 'surface temperature': 42.0}
{'air temperature': 18.0,
 'date': '15/12/2023',
 'max wind speed': 14.0,
 'relative humidity': 52.0,
 'surface temperature': 36.0}
{'air temperature': 18.0,
 'date': '15/12/2023',
 'max wind speed': 14.0,
 'relative humidity': 52.0,
 'surface temperature': 38.0}
{'air temperature': 18.0,
 'date': '15/12/2023',
 'max wind speed': 14.0,
 'relative humidity': 52.0,
 'surface temperature': 40.0}
{'air temperature': 18.0,
 'date': '16/12/2023',
 'max wind speed': 13.0,
 'relative humidity': 53.7,
 'surface temperature': 43.0}
{'air temperature': 18.0,
 'date': '16/12/2023',
 'max wind speed': 13.0,
 'relative humidity': 53.7,
 'surface temperature': 33.0}
{'air temperature': 18.0,
 'date': '16/12/2023',
 'max wind speed': 13.0,
 'relative humidity': 53.7,
 'surface temperature': 54.0}
{'air temperature': 18.0,
 'date': '16/12/2023',
 'max wind speed': 13.0,
 '

In [28]:
#2(d)
result2d=climate_db.aggregate([{'$unwind': '$hotspot'},
    {'$project': {"_id":0,'date time':'$hotspot.datetime','air temperature': '$air_temperature_celcius','surface temperature': '$hotspot.surface_temperature_celcius','confidence':'$hotspot.confidence'}}
                                     ,{'$match': {'confidence': {'$gt': 80,'$lte': 100}}},
                                    ])
for doc in result2d:
   print(doc)



{'date time': '2023-03-06 05:06:30', 'air temperature': 20.0, 'surface temperature': 62.0, 'confidence': 87.0}
{'date time': '2023-03-06 05:06:20', 'air temperature': 20.0, 'surface temperature': 59.0, 'confidence': 85.0}
{'date time': '2023-03-07 04:16:10', 'air temperature': 19.0, 'surface temperature': 64.0, 'confidence': 88.0}
{'date time': '2023-03-09 13:23:40', 'air temperature': 23.0, 'surface temperature': 41.0, 'confidence': 86.0}
{'date time': '2023-03-10 04:48:40', 'air temperature': 19.0, 'surface temperature': 105.0, 'confidence': 100.0}
{'date time': '2023-03-10 04:46:20', 'air temperature': 19.0, 'surface temperature': 109.0, 'confidence': 100.0}
{'date time': '2023-03-10 04:45:30', 'air temperature': 19.0, 'surface temperature': 73.0, 'confidence': 94.0}
{'date time': '2023-03-10 04:45:30', 'air temperature': 19.0, 'surface temperature': 80.0, 'confidence': 97.0}
{'date time': '2023-03-10 04:43:50', 'air temperature': 19.0, 'surface temperature': 55.0, 'confidence': 81.

In [29]:
#2(e)
result2e = climate_db.aggregate([
    {'$unwind': '$hotspot'},
    {'$sort': {'hotspot.surface_temperature_celsius': -1}},
    {'$project':{'_id':0}}, {'$limit': 10}
])

for doc in result2e:
    print(doc)

{'station': '948701', 'date': '10/3/2023', 'air_temperature_celcius': 19.0, 'relative_humidity': 60.2, 'windspeed_knots': 6.4, 'max_wind_speed': 12.0, 'precipitation': '0.00I', 'GHI_w/m2': 149.0, 'hotspot': {'latitude': -37.223, 'longitude': 147.9431, 'datetime': '2023-03-10 04:48:40', 'confidence': 100.0, 'date': '10/3/2023', 'surface_temperature_celcius': 105.0}}
{'station': '948701', 'date': '10/3/2023', 'air_temperature_celcius': 19.0, 'relative_humidity': 60.2, 'windspeed_knots': 6.4, 'max_wind_speed': 12.0, 'precipitation': '0.00I', 'GHI_w/m2': 149.0, 'hotspot': {'latitude': -37.2252, 'longitude': 147.9363, 'datetime': '2023-03-10 04:46:20', 'confidence': 100.0, 'date': '10/3/2023', 'surface_temperature_celcius': 109.0}}
{'station': '948701', 'date': '8/3/2023', 'air_temperature_celcius': 21.0, 'relative_humidity': 51.7, 'windspeed_knots': 7.2, 'max_wind_speed': 13.0, 'precipitation': '0.00I', 'GHI_w/m2': 177.0, 'hotspot': {'latitude': -38.1031, 'longitude': 142.4797, 'datetime':

In [18]:
#2(f)

result2f = climate_db.aggregate([{'$project': {'date': 1, 'hotspot': 1}},
    {'$group': {
        '_id': '$date',  
        'frequency': {'$sum': {'$size': '$hotspot'}}}} , 
                                 {'$sort': {'date': 1}}, {'$project': {'_id': 0,'date': '$_id','frequency of fire per day': '$frequency'}}
])

for doc in result2f:
    print(doc)

{'date': '26/11/2023', 'frequency of fire per day': 0}
{'date': '7/1/2023', 'frequency of fire per day': 0}
{'date': '13/7/2023', 'frequency of fire per day': 0}
{'date': '26/12/2023', 'frequency of fire per day': 0}
{'date': '5/6/2023', 'frequency of fire per day': 0}
{'date': '4/7/2023', 'frequency of fire per day': 1}
{'date': '17/12/2023', 'frequency of fire per day': 0}
{'date': '12/1/2023', 'frequency of fire per day': 0}
{'date': '19/5/2023', 'frequency of fire per day': 0}
{'date': '4/4/2023', 'frequency of fire per day': 89}
{'date': '14/6/2023', 'frequency of fire per day': 4}
{'date': '27/7/2023', 'frequency of fire per day': 0}
{'date': '27/9/2023', 'frequency of fire per day': 7}
{'date': '20/12/2023', 'frequency of fire per day': 0}
{'date': '6/3/2023', 'frequency of fire per day': 2}
{'date': '9/3/2023', 'frequency of fire per day': 3}
{'date': '10/12/2023', 'frequency of fire per day': 2}
{'date': '5/4/2023', 'frequency of fire per day': 49}
{'date': '21/2/2023', 'frequ

In [19]:
result2g=  climate_db.aggregate([ {'$unwind': '$hotspot'},
                                 {'$project':{'hotspot':1}},
                                 {'$match': {'hotspot.confidence': {'$lt': 70}}}, 
                                 {'$project':{'_id':0}}
                                 
])


for doc in result2g:
    print(doc)

{'hotspot': {'latitude': -37.7885, 'longitude': 141.9352, 'datetime': '2023-03-08 04:51:00', 'confidence': 68.0, 'date': '8/3/2023', 'surface_temperature_celcius': 55.0}}
{'hotspot': {'latitude': -37.7171, 'longitude': 147.5866, 'datetime': '2023-03-09 03:57:00', 'confidence': 54.0, 'date': '9/3/2023', 'surface_temperature_celcius': 44.0}}
{'hotspot': {'latitude': -36.2544, 'longitude': 148.0353, 'datetime': '2023-03-10 04:43:00', 'confidence': 55.0, 'date': '10/3/2023', 'surface_temperature_celcius': 42.0}}
{'hotspot': {'latitude': -37.2197, 'longitude': 147.9621, 'datetime': '2023-03-10 04:42:30', 'confidence': 54.0, 'date': '10/3/2023', 'surface_temperature_celcius': 43.0}}
{'hotspot': {'latitude': -37.0286, 'longitude': 148.155, 'datetime': '2023-03-13 23:58:50', 'confidence': 56.0, 'date': '13/3/2023', 'surface_temperature_celcius': 42.0}}
{'hotspot': {'latitude': -37.0316, 'longitude': 148.1519, 'datetime': '2023-03-13 12:57:00', 'confidence': 52.0, 'date': '13/3/2023', 'surface_

In [20]:
result2h = climate_db.aggregate([{'$project': {'date': 1, 'hotspot': 1}},
    {'$unwind': '$hotspot'},
    {'$group': {
        '_id': '$date',  
        'frequency': {'$avg': '$hotspot.surface_temperature_celcius'}}} , {'$sort': {'_id': 1}},
        {'$project': {'_id': 0,'date': '$_id','average temperature per day': {'$round': ['$frequency', 2]}}}
])

for doc in result2h:
    print(doc)

{'date': '1/10/2023', 'average temperature per day': 48.25}
{'date': '1/4/2023', 'average temperature per day': 46.71}
{'date': '1/5/2023', 'average temperature per day': 68.4}
{'date': '1/6/2023', 'average temperature per day': 54.0}
{'date': '1/7/2023', 'average temperature per day': 30.5}
{'date': '1/8/2023', 'average temperature per day': 58.0}
{'date': '10/10/2023', 'average temperature per day': 53.33}
{'date': '10/12/2023', 'average temperature per day': 46.0}
{'date': '10/3/2023', 'average temperature per day': 69.38}
{'date': '10/5/2023', 'average temperature per day': 52.87}
{'date': '10/8/2023', 'average temperature per day': 63.0}
{'date': '10/9/2023', 'average temperature per day': 56.5}
{'date': '11/11/2023', 'average temperature per day': 46.25}
{'date': '11/4/2023', 'average temperature per day': 46.29}
{'date': '11/5/2023', 'average temperature per day': 59.37}
{'date': '11/6/2023', 'average temperature per day': 41.5}
{'date': '12/11/2023', 'average temperature per da

In [21]:
result2h = climate_db.aggregate([
    {'$project':{ 'hotspot': 0}},
    {'$sort': {'GHI_w/m2': 1}},
    {'$limit': 10}, {'$project':{'_id':0}}
])

for doc in result2h:
    print(doc)

{'station': '948701', 'date': '2/8/2023', 'air_temperature_celcius': 5.0, 'relative_humidity': 38.6, 'windspeed_knots': 1.8, 'max_wind_speed': 5.1, 'precipitation': '0.00I', 'GHI_w/m2': 47.0}
{'station': '948701', 'date': '30/6/2023', 'air_temperature_celcius': 5.0, 'relative_humidity': 34.9, 'windspeed_knots': 4.3, 'max_wind_speed': 11.1, 'precipitation': '0.00A', 'GHI_w/m2': 48.0}
{'station': '948701', 'date': '1/7/2023', 'air_temperature_celcius': 5.0, 'relative_humidity': 33.1, 'windspeed_knots': 5.8, 'max_wind_speed': 14.0, 'precipitation': '0.00I', 'GHI_w/m2': 49.0}
{'station': '948701', 'date': '11/7/2023', 'air_temperature_celcius': 6.0, 'relative_humidity': 39.0, 'windspeed_knots': 1.8, 'max_wind_speed': 5.1, 'precipitation': '0.01G', 'GHI_w/m2': 56.0}
{'station': '948701', 'date': '15/7/2023', 'air_temperature_celcius': 6.0, 'relative_humidity': 37.1, 'windspeed_knots': 5.8, 'max_wind_speed': 9.9, 'precipitation': '0.00I', 'GHI_w/m2': 57.0}
{'station': '948701', 'date': '2/7/

In [22]:
result2i = climate_db.aggregate([
    {'$project':{ 'hotspot': 0}},
    {'$match': {'precipitation': {'$regex': '^(0\\.2[0-9]|0\\.3[0-5])[A-I]?$'}}},
     {'$project':{'_id':0,'hotspot':0}}
])

for doc in result2i:
    print(doc)


{'station': '948700', 'date': '13/1/2023', 'air_temperature_celcius': 19.0, 'relative_humidity': 54.1, 'windspeed_knots': 11.2, 'max_wind_speed': 18.1, 'precipitation': '0.31G', 'GHI_w/m2': 157.0}
{'station': '948701', 'date': '29/3/2023', 'air_temperature_celcius': 17.0, 'relative_humidity': 49.9, 'windspeed_knots': 12.2, 'max_wind_speed': 21.0, 'precipitation': '0.24G', 'GHI_w/m2': 146.0}
{'station': '948701', 'date': '20/4/2023', 'air_temperature_celcius': 20.0, 'relative_humidity': 53.5, 'windspeed_knots': 7.2, 'max_wind_speed': 15.9, 'precipitation': '0.31G', 'GHI_w/m2': 166.0}
{'station': '948701', 'date': '26/4/2023', 'air_temperature_celcius': 11.0, 'relative_humidity': 40.8, 'windspeed_knots': 12.2, 'max_wind_speed': 20.0, 'precipitation': '0.24G', 'GHI_w/m2': 102.0}
{'station': '948701', 'date': '27/5/2023', 'air_temperature_celcius': 14.0, 'relative_humidity': 48.1, 'windspeed_knots': 13.7, 'max_wind_speed': 19.0, 'precipitation': '0.35G', 'GHI_w/m2': 122.0}
{'station': '948

### Task 2c
to improve the congestion, we need to give a unique index that will be query a lot, so given the date, is important in both climate and hotspot report, whjhich we will create key index for date in ascending order to improve the query

In [35]:
result = climate_db.create_index([('date', pymongo.ASCENDING)], unique=True)
climate_db.drop_index(result)
